## RNN to Make a Model Learn Addition and Subtraction

## Model 2 : Encoder-Decoder using RepeatVector

This is a many-to-many sequence-to-sequence (seq2seq) model where the input and output sequences have <br>
the same length. Unlike the traditional encoder-decoder structure (where the encoder compresses an <br>
input into a single latent vector), this model passes the entire sequence through both RNNs.<br>


Given the string "54+7", the model should return a prediction: "61" <br>
Given the string "54-3", the model should return a prediction: "51" <br>

We will use an encoder-decoder architecture



### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector, Input,LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.callbacks import Callback

from termcolor import colored

import sys
sys.stderr = open('err.txt', 'w')

2025-03-09 18:40:37.109149: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 18:40:37.116521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 18:40:37.124844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 18:40:37.127371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 18:40:37.133855: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

### Generate Data

In [3]:
all_chars    = '0123456789+-'
num_features = len(all_chars)
print('Number of features:', num_features)
trace = True
target_sequence_length = 5

Number of features: 12


In [4]:

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))



In [5]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    add = np.squeeze(np.random.randint(low=0, high=100)) > 50.
    if add:
        example = str(first_num) + '+' + str(second_num)
        label = str(first_num+second_num)
    else:
        example = str(first_num) + '-' + str(second_num)
        label = str(first_num-second_num)
    return example, label

generate_data()

('42+36', '78')

### Support Functions

### Create the Model

In [6]:
"""

Interesting callback that prints the accuracy on each time step

"""
class SimpleLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Val Accuracy = {logs.get('val_accuracy', logs.get('val_acc', 'N/A')):.4f}")

simple_logger = SimpleLogger()

In [7]:
# Network Hyperparameters

hidden_units = 64
num_features = 12

model = Sequential([
    SimpleRNN(hidden_units, return_sequences=False, input_shape=(None, num_features)),  # Encoder (produces bottleneck vector)                                           # Bottleneck layer
    RepeatVector(target_sequence_length),                                               # Expands bottleneck to match output sequence length
    SimpleRNN(hidden_units, return_sequences=True),                                     # Decoder RNN
    Dense(num_features, activation='softmax')                                           # Final output layer
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 256)            │        68,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 5, 256)         │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5, 12)          │         3,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,276 (794.05 KB)

 Trainable params: 203,276 (794.05 KB)

 Non-trainable params: 0 (0.00 B)

### Vectorize and De-Vectorize Data

In [8]:
def vectorize_example(example, label):
    
    x = np.zeros((target_sequence_length, num_features))
    y = np.zeros((target_sequence_length, num_features))
    
    diff_x = target_sequence_length - len(example)
    diff_y = target_sequence_length - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 76-66 10
Vectorized Example and Label Shapes: (5, 12) (5, 12)


In [9]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

def strip_padding(example):
    encountered_non_zero = False
    output = ''
    for c in example:
        if not encountered_non_zero and c == '0':
            continue
        if c == '+' or c == '-':
            encountered_non_zero = False
        else:
            encountered_non_zero = True
        output += c
    return output

devectorize_example(x)

'76-66'

In [10]:
print(devectorize_example(y), ':', strip_padding(devectorize_example(y)))

00010 : 10


### Create Dataset

In [11]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, target_sequence_length, num_features))
    y_train = np.zeros((num_examples, target_sequence_length, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset(20000)
print(x_train.shape, y_train.shape)

(20000, 5, 12) (20000, 5, 12)


In [12]:
devectorize_example(x_train[0])

'32+28'

In [13]:
devectorize_example(y_train[0])

'00060'

### Training the Model

In [14]:
def see_trained_output (preds):
    full_seq_acc = 0
    for i, pred in enumerate(preds):
        pred_str   = strip_padding(devectorize_example(pred))
        y_test_str = strip_padding(devectorize_example(y_test[i]))
        x_test_str = strip_padding(devectorize_example(x_test[i]))
        col = 'green' if pred_str == y_test_str else 'red'
        full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
        outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
        print(colored(outstring, col))
    print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

In [15]:
""" 
    We train the model and see a random list of 20 examples how it is working 
    you will see the progression of the learning, but only if trace is True
"""
trace = True
for iteration in range(1,200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, epochs=1, validation_split=0.2, verbose=False,
         batch_size=1024, callbacks=[simple_logger])
    x_test, y_test = create_dataset(num_examples=20)
    preds = model.predict(x_test)
    if trace:
        see_trained_output(preds)


--------------------------------------------------
Iteration 1
Epoch 1: Loss = 1.3517, Val Accuracy = 0.6040
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Input: 42-26, Out: 16, Pred: 2
Input: 90-85, Out: 5, Pred: 11
Input: 98-74, Out: 24, Pred: 11
Input: 11-, Out: 11, Pred: 8
Input: 10+81, Out: 91, Pred: 18
Input: 28+74, Out: 102, Pred: 18
Input: 29+6, Out: 35, Pred: 22
Input: 42-20, Out: 22, Pred: 3
Input: 52-13, Out: 39, Pred: 2
Input: 20+47, Out: 67, Pred: 18
Input: 60+68, Out: 128, Pred: 111
Input: 97+21, Out: 118, Pred: 111
Input: 71+61, Out: 132, Pred: 111
Input: 18-83, Out: -65, Pred: -2
Input: 77+88, Out: 165, Pred: 111
Input: 51+65, Out: 116, Pred: 111
Input: 12-67, Out: -55, Pred: -22
Input: 49+21, Out: 70, Pred: 11
Input: 56-33, Out: 23, Pred: 2
Input: 16-2, Out: 14, Pred: 18

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 2
Epoch 1: Loss = 1.0097, Val Accuracy = 0.6325
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Input: 13-61, Out: -48, Pre

Epoch 1: Loss = 0.7595, Val Accuracy = 0.7199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Input: 3+67, Out: 70, Pred: 74
Input: 32-52, Out: -20, Pred: -26
Input: 74-99, Out: -25, Pred: -24
Input: 64-21, Out: 43, Pred: 45
Input: 58+27, Out: 85, Pred: 84
Input: 79+40, Out: 119, Pred: 125
Input: 25-42, Out: -17, Pred: -24
Input: 35-57, Out: -22, Pred: -20
Input: 67+5, Out: 72, Pred: 66
Input: 55+16, Out: 71, Pred: 74
Input: 46-62, Out: -16, Pred: -16
Input: 29+84, Out: 113, Pred: 113
Input: 9+20, Out: 29, Pred: 63
Input: 90-49, Out: 41, Pred: 34
Input: 56+67, Out: 123, Pred: 136
Input: 52+93, Out: 145, Pred: 147
Input: 86-56, Out: 30, Pred: 26
Input: 20-87, Out: -67, Pred: -66
Input: 76-16, Out: 60, Pred: 50
Input: 14-90, Out: -76, Pred: -70

Full sequence accuracy: 10.000 %

--------------------------------------------------
Iteration 10
Epoch 1: Loss = 0.7286, Val Accuracy = 0.7369
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Input: 62-46, Out: 16, Pred: 11
Input: 65-82, Out: -17, Pred: -18
Input: 4

Epoch 1: Loss = 0.5660, Val Accuracy = 0.7997
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Input: 64-47, Out: 17, Pred: 14
Input: 40-40, Out: , Pred: 1
Input: 64-84, Out: -20, Pred: -19
Input: 71+98, Out: 169, Pred: 169
Input: 81-94, Out: -13, Pred: -16
Input: 29-69, Out: -40, Pred: -40
Input: 26-3, Out: 23, Pred: 20
Input: 76-40, Out: 36, Pred: 35
Input: 60-50, Out: 10, Pred: 1
Input: 26-54, Out: -28, Pred: -20
Input: 13+81, Out: 94, Pred: 97
Input: 35-96, Out: -61, Pred: -61
Input: 8-86, Out: -78, Pred: -79
Input: 50-35, Out: 15, Pred: 16
Input: 72+32, Out: 104, Pred: 103
Input: 96+37, Out: 133, Pred: 133
Input: -24, Out: -24, Pred: -26
Input: 86-95, Out: -9, Pred: -1
Input: 60+11, Out: 71, Pred: 71
Input: 75-30, Out: 45, Pred: 44

Full sequence accuracy: 25.000 %

--------------------------------------------------
Iteration 18
Epoch 1: Loss = 0.5307, Val Accuracy = 0.8073
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 84+87, Out: 171, Pred: 160
Input: 14-71, Out: -57, Pred: -55
Input: 36+16,

Epoch 1: Loss = 0.2686, Val Accuracy = 0.9251
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Input: 62-61, Out: 1, Pred: 1
Input: 25+36, Out: 61, Pred: 60
Input: 8-27, Out: -19, Pred: -19
Input: 33+68, Out: 101, Pred: 101
Input: 25-67, Out: -42, Pred: -41
Input: 27-38, Out: -11, Pred: -11
Input: 49-65, Out: -16, Pred: -16
Input: 64-18, Out: 46, Pred: 46
Input: 67+85, Out: 152, Pred: 152
Input: 60-6, Out: 54, Pred: 54
Input: 4-97, Out: -93, Pred: -83
Input: 57-78, Out: -21, Pred: -20
Input: 60+78, Out: 138, Pred: 138
Input: 30+42, Out: 72, Pred: 72
Input: 76-67, Out: 9, Pred: 1
Input: 17-72, Out: -55, Pred: -55
Input: 19+15, Out: 34, Pred: 34
Input: 52-95, Out: -43, Pred: -43
Input: 73-18, Out: 55, Pred: 56
Input: 45+50, Out: 95, Pred: 95

Full sequence accuracy: 70.000 %

--------------------------------------------------
Iteration 26
Epoch 1: Loss = 0.2389, Val Accuracy = 0.9298
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 76-74, Out: 2, Pred: 2
Input: 95-93, Out: 2, Pred: 2
Input: 43+72, Out:

Epoch 1: Loss = 0.1107, Val Accuracy = 0.9768
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 68+77, Out: 145, Pred: 145
Input: 85-92, Out: -7, Pred: -7
Input: 37+64, Out: 101, Pred: 101
Input: 56-89, Out: -33, Pred: -33
Input: 28+99, Out: 127, Pred: 127
Input: 77+6, Out: 83, Pred: 83
Input: 32+12, Out: 44, Pred: 44
Input: 18+81, Out: 99, Pred: 9
Input: 37-15, Out: 22, Pred: 22
Input: 38-10, Out: 28, Pred: 28
Input: 34+, Out: 34, Pred: 34
Input: 31-38, Out: -7, Pred: -7
Input: 14+80, Out: 94, Pred: 94
Input: 24+55, Out: 79, Pred: 89
Input: 79-11, Out: 68, Pred: 68
Input: 37+59, Out: 96, Pred: 96
Input: 80+40, Out: 120, Pred: 120
Input: 72+9, Out: 81, Pred: 81
Input: 91-26, Out: 65, Pred: 65
Input: 77+36, Out: 113, Pred: 113

Full sequence accuracy: 90.000 %

--------------------------------------------------
Iteration 34
Epoch 1: Loss = 0.0934, Val Accuracy = 0.9811
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Input: 37-78, Out: -41, Pred: -41
Input: 43-51, Out: -8, Pred: -8
Input: 28-6, Out: 22

...

Epoch 1: Loss = 0.0012, Val Accuracy = 0.9981
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 16-93, Out: -77, Pred: -77
Input: 61+45, Out: 106, Pred: 106
Input: 14+63, Out: 77, Pred: 77
Input: 54+41, Out: 95, Pred: 95
Input: 93+43, Out: 136, Pred: 136
Input: 26+67, Out: 93, Pred: 93
Input: 5-5, Out: , Pred: 
Input: 13-42, Out: -29, Pred: -29
Input: 13+, Out: 13, Pred: 13
Input: 30+41, Out: 71, Pred: 71
Input: 79-43, Out: 36, Pred: 36
Input: 11+52, Out: 63, Pred: 63
Input: 49+69, Out: 118, Pred: 118
Input: 33-16, Out: 17, Pred: 17
Input: 49+1, Out: 50, Pred: 50
Input: 98-46, Out: 52, Pred: 52
Input: 14-2, Out: 12, Pred: 12
Input: 60-73, Out: -13, Pred: -13
Input: 43-62, Out: -19, Pred: -19
Input: 57+29, Out: 86, Pred: 86

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 162
Epoch 1: Loss = 0.0012, Val Accuracy = 0.9982
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 65+63, Out: 128, Pred: 128
Input: 53+21, Out: 74, Pred: 74
Input: 20+76, Out: 96

Epoch 1: Loss = 0.0010, Val Accuracy = 0.9980
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Input: 74-39, Out: 35, Pred: 35
Input: 57+62, Out: 119, Pred: 119
Input: 67+37, Out: 104, Pred: 104
Input: 4-91, Out: -87, Pred: -87
Input: 65-16, Out: 49, Pred: 49
Input: 71+57, Out: 128, Pred: 128
Input: 28-42, Out: -14, Pred: -14
Input: 99-24, Out: 75, Pred: 75
Input: 77+31, Out: 108, Pred: 108
Input: 79+69, Out: 148, Pred: 148
Input: 83-32, Out: 51, Pred: 51
Input: 86+90, Out: 176, Pred: 176
Input: 63+55, Out: 118, Pred: 118
Input: 85+80, Out: 165, Pred: 165
Input: 25+74, Out: 99, Pred: 99
Input: 96-12, Out: 84, Pred: 84
Input: 39-97, Out: -58, Pred: -58
Input: 25-35, Out: -10, Pred: -10
Input: 11-44, Out: -33, Pred: -33
Input: 6-8, Out: -2, Pred: -2

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 170
Epoch 1: Loss = 0.0010, Val Accuracy = 0.9983
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 82+43, Out: 125, Pred: 125
Input: 50+27, Out: 77, Pred: 77
In

Epoch 1: Loss = 0.0009, Val Accuracy = 0.9982
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 59-34, Out: 25, Pred: 25
Input: 74+42, Out: 116, Pred: 116
Input: 76-98, Out: -22, Pred: -22
Input: 19-88, Out: -69, Pred: -69
Input: 16-77, Out: -61, Pred: -61
Input: 10+75, Out: 85, Pred: 85
Input: 91-90, Out: 1, Pred: 
Input: 35-96, Out: -61, Pred: -61
Input: 92+58, Out: 150, Pred: 150
Input: 28-1, Out: 27, Pred: 27
Input: 8+60, Out: 68, Pred: 68
Input: +9, Out: 9, Pred: 9
Input: 77+53, Out: 130, Pred: 130
Input: 63+54, Out: 117, Pred: 117
Input: 80-9, Out: 71, Pred: 71
Input: 83-27, Out: 56, Pred: 56
Input: 20-39, Out: -19, Pred: -19
Input: 94+74, Out: 168, Pred: 168
Input: 90-16, Out: 74, Pred: 74
Input: 36+24, Out: 60, Pred: 60

Full sequence accuracy: 95.000 %

--------------------------------------------------
Iteration 178
Epoch 1: Loss = 0.0009, Val Accuracy = 0.9983
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 59+16, Out: 75, Pred: 75
Input: 47-85, Out: -38, Pred: -38
Input: 61-74, Out

Epoch 1: Loss = 0.0008, Val Accuracy = 0.9983
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Input: 71-53, Out: 18, Pred: 18
Input: 87+42, Out: 129, Pred: 129
Input: 31-62, Out: -31, Pred: -31
Input: 69-12, Out: 57, Pred: 57
Input: 89-35, Out: 54, Pred: 54
Input: 60+67, Out: 127, Pred: 127
Input: 17-84, Out: -67, Pred: -67
Input: 12-46, Out: -34, Pred: -34
Input: 69+66, Out: 135, Pred: 135
Input: 50-29, Out: 21, Pred: 21
Input: 46-26, Out: 20, Pred: 20
Input: 6-34, Out: -28, Pred: -28
Input: 39-2, Out: 37, Pred: 37
Input: 91-15, Out: 76, Pred: 76
Input: 78+28, Out: 106, Pred: 106
Input: 75+35, Out: 110, Pred: 110
Input: 32+19, Out: 51, Pred: 51
Input: 43-45, Out: -2, Pred: -2
Input: 2-67, Out: -65, Pred: -65
Input: 75-20, Out: 55, Pred: 55

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 186
Epoch 1: Loss = 0.0008, Val Accuracy = 0.9981
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 1-50, Out: -49, Pred: -49
Input: 46+34, Out: 80, Pred: 80
Input: 34

Epoch 1: Loss = 0.0007, Val Accuracy = 0.9980
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Input: 36+49, Out: 85, Pred: 85
Input: 80+11, Out: 91, Pred: 91
Input: 52-67, Out: -15, Pred: -15
Input: 7-6, Out: 1, Pred: 1
Input: 9-57, Out: -48, Pred: -48
Input: 83+55, Out: 138, Pred: 138
Input: 70+87, Out: 157, Pred: 157
Input: 59-29, Out: 30, Pred: 30
Input: 93+18, Out: 111, Pred: 111
Input: 56-85, Out: -29, Pred: -29
Input: 68+64, Out: 132, Pred: 132
Input: 85-60, Out: 25, Pred: 25
Input: 74+39, Out: 113, Pred: 113
Input: 72-48, Out: 24, Pred: 24
Input: 25+57, Out: 82, Pred: 82
Input: 92-91, Out: 1, Pred: 1
Input: 44+58, Out: 102, Pred: 102
Input: 44+19, Out: 63, Pred: 63
Input: 14+23, Out: 37, Pred: 37
Input: 93-2, Out: 91, Pred: 92

Full sequence accuracy: 95.000 %

--------------------------------------------------
Iteration 194
Epoch 1: Loss = 0.0007, Val Accuracy = 0.9982
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Input: 72+64, Out: 136, Pred: 136
Input: 34-33, Out: 1, Pred: 1
Input: 12-15, Out:

In [16]:
import session_info
session_info.show(html=False)

-----
keras               3.6.0
numpy               1.26.4
session_info        1.0.0
tensorflow          2.17.1
termcolor           NA
-----
IPython             8.28.0
jupyter_client      8.6.3
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2025-03-09 18:43
